In [ ]:
import pandas as pd

In [ ]:
car = pd.read_csv('quikr_car.csv')
car.head()

In [ ]:
car.shape

In [ ]:
car.info()

Some points about data

In [ ]:
car['year'].unique() #many garbage values 

In [ ]:
car['Price'].unique() #there is ask for price which is a garbage value

In [ ]:
car['company'].unique() #some garbage values here as well

In [ ]:
car['fuel_type'].unique() #nan value

In [ ]:
car['kms_driven'].unique() #kms after each value which makes it an object data type

In [ ]:
car['name'].unique() #needs to be standardized

Data Cleaning

In [ ]:
backup1 = car.copy()

In [ ]:
backup1 = backup1[backup1['year'].str.isnumeric()]

In [ ]:
backup1['year'] = backup1['year'].astype(int)
backup1.info()

In [ ]:
backup1 = backup1[backup1['Price']!="Ask For Price"]
backup1['Price'].unique()

In [ ]:
backup1['Price'] = backup1['Price'].str.replace(',','').astype(int)

In [ ]:
backup1.info()

In [ ]:
backup1['kms_driven'].unique()

In [ ]:
backup1['kms_driven'] = backup1['kms_driven'].str.split(' ').str.get(0).str.replace(',','')

In [ ]:
backup1 = backup1[backup1['kms_driven'].str.isnumeric()]

In [ ]:
backup1['kms_driven'].unique()

In [ ]:
backup1['kms_driven'] = backup1['kms_driven'].astype(int)

In [ ]:
backup1.info()

In [ ]:
backup1 = backup1[~backup1['fuel_type'].isna()]

In [ ]:
backup1.info()

In [ ]:
backup1['name'] = backup1['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [ ]:
backup1.info()

In [ ]:
backup1 = backup1.reset_index(drop=True)

In [ ]:
backup1.describe()

In [ ]:
backup1 = backup1[backup1['Price']<6e6].reset_index(drop=True)

In [ ]:
backup1

In [ ]:
backup1.to_csv('cleaned_data.csv')

Building the model

In [ ]:
x = backup1.drop(columns='Price')
y = backup1['Price']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [ ]:
ohe = OneHotEncoder()
ohe.fit(x[['name', 'company', 'fuel_type']])

In [ ]:
ohe.categories_

In [ ]:
column_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']),remainder='passthrough')

In [ ]:
lr = LinearRegression()

In [ ]:
pipe = make_pipeline(column_trans,lr)

In [ ]:
pipe.fit(x_train, y_train)

In [ ]:
y_pred = pipe.predict(x_test)

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
score = []
for i in range (1,1000):
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=i)
    lr = LinearRegression()
    pipe = make_pipeline(column_trans,lr)
    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)
    score.append(r2_score(y_test, y_pred))

In [ ]:
import numpy as np
np.argmax(score)
score[np.argmax(score)] #by this it is inferred that at random state 432, the model has the highest r2 score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=293)
lr = LinearRegression()
pipe = make_pipeline(column_trans,lr)
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)
r2_score(y_test, y_pred)

In [ ]:
import pickle
pickle.dump(pipe, open('LinearRegressionModel.pkl','wb'))

In [ ]:
pipe.predict(pd.DataFrame([['Maruti Suzuki Swift', 'Maruti', 2019, 100, 'Petrol']], columns=['name','company','year','kms_driven','fuel_type']))